In [ ]:
!pip install gradio


INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 18.1/18.1 MB 11.2 MB/s eta 0:00:00
   ---------------------------------------- 11.4/11.4 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.11
    Uninstalling urllib3-1.26.11:
      Successfully uninstalled urllib3-1.26.11
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.11.1
    Uninstalling tomlkit-0.11.1:
      Successfully uninstalled tomlkit-0.11.1
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
botocore 1.27.28 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.3.0 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [3]:
!pip install assemblyai

In [5]:
!pip install translate

In [ ]:

import gradio as gr
import assemblyai as aai
from translate import Translator
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import uuid
from pathlib import Path

def voice_to_voice(audio_file):
    
    #transcribe audio
    transcription_response = audio_transcription(audio_file)

    if transcription_response.status == aai.TranscriptStatus.error:
        raise gr.Error(transcription_response.error)
    else:
        text = transcription_response.text

    es_translation, tr_translation, ja_translation = text_translation(text)

    es_audi_path = text_to_speech(es_translation)
    tr_audi_path = text_to_speech(tr_translation)
    ja_audi_path = text_to_speech(ja_translation)

    es_path = Path(es_audi_path)
    tr_path = Path(tr_audi_path)
    ja_path = Path(ja_audi_path)

    return es_path, tr_path, ja_path


def audio_transcription(audio_file):

    aai.settings.api_key = "c4fae7dfe3a6460b84e47b66fc12c530"

    transcriber = aai.Transcriber()
    transcription = transcriber.transcribe(audio_file)

    return transcription

def text_translation(text):
    
    translator_es = Translator(from_lang="en", to_lang="es")
    es_text = translator_es.translate(text)

    translator_tr = Translator(from_lang="en", to_lang="tr")
    tr_text = translator_tr.translate(text)

    translator_ja = Translator(from_lang="en", to_lang="ja")
    ja_text = translator_ja.translate(text)

    return es_text, tr_text, ja_text

def text_to_speech(text):

    client = ElevenLabs(
        api_key= "sk_02a3c1a9a18d279709f67fc9b786071920ffcdf42f03d5d4",
    )

    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="2EiwWnXFnvU5JabPnv8n", #clone your voice on elevenlabs dashboard and copy the id
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_multilingual_v2", # use the turbo model for low latency, for other languages use the `eleven_multilingual_v2`
        voice_settings=VoiceSettings(
            stability=0.5,
            similarity_boost=0.8,
            style=0.5,
            use_speaker_boost=True,
        ),
    )

    # Generating a unique file name for the output MP3 file
    save_file_path = f"{uuid.uuid4()}.mp3"

    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{save_file_path}: A new audio file was saved successfully!")

    # Return the path of the saved audio file
    return save_file_path

audio_input = gr.Audio(
    sources=["microphone"],
    type="filepath"
)

demo = gr.Interface(
    fn=voice_to_voice,
    inputs=audio_input,
    outputs=[gr.Audio(label="Spanish"), gr.Audio(label="Turkish"), gr.Audio(label="Japanese")]
)

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


f5a17d7b-930d-487a-a2f9-20bcf6b6dd56.mp3: A new audio file was saved successfully!
c3649c79-4c30-42ec-9079-f352ed06157b.mp3: A new audio file was saved successfully!
2b77bb20-c2c4-4b97-a5af-149b39353841.mp3: A new audio file was saved successfully!


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\HP\anaconda3\New folder\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(

b0654e76-2976-44bd-9a28-420f0ae80b36.mp3: A new audio file was saved successfully!
fa333ffd-77dd-4ebf-9ad1-8fdfda770a7b.mp3: A new audio file was saved successfully!
e2e1b873-efa4-4575-ae5c-dd97f85964ad.mp3: A new audio file was saved successfully!
e899dc05-3cc0-49b1-a75f-62e255bb92bd.mp3: A new audio file was saved successfully!
ab909b44-949e-4cc9-92b5-d0994dae66c8.mp3: A new audio file was saved successfully!
ecd29af5-7583-4206-9d00-022ad63ed24c.mp3: A new audio file was saved successfully!


In [6]:
!pip install elevenlabs